In [2]:
#Import package and data
import numpy as np
import pandas as pd
import matplotlib 
import seaborn as sns
from matplotlib import pyplot as plt

dataset=pd.read_csv('winequality-white.csv')
dataset_1=pd.read_csv('winequality-red.csv')
pd.options.mode.chained_assignment = None 

In [3]:

dataset['Kind']='White'
dataset_1['Kind']='Red'
dataset=pd.concat([dataset,dataset_1])

In [4]:
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Kind
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,White
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,White
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,White
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,White
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,White


In [5]:
dummies=pd.get_dummies(dataset['Kind'])
dataset=dataset.join(dummies)
dataset.drop(['Kind'], axis=1,inplace=True)
dataset=dataset[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol','Red','White','quality']]
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Red,White,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0,1,6
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,1,0,6
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0,1,5
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,1,0,5
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0,1,6


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[:, 1:13].values, dataset.quality.values, test_size = 0.2, random_state = 0)

In [7]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [8]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', max_depth=10,random_state = 0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_test, y_pred)
print('******Test set confusion matrix************')
print(cm_test)


******Test set confusion matrix************
[[  4   0   4   2   0   0]
 [  0  16  38  10   1   0]
 [  0   0 501 143   2   0]
 [  0   3 129 644  55   0]
 [  0   0  12 155 173   1]
 [  0   0   0  24  11  11]]


In [9]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [5, 10, 15,20], 'max_features': [1,2,3,4,5], 'max_depth':[5,10,15],'min_samples_split':[2,5,10],'min_samples_leaf':[1,5,10]},
    {'bootstrap': [False], 'n_estimators': [5, 10, 20], 'max_features': [1,2,3,4,5],'max_depth':[5,10,15],'min_samples_split':[2,5,10],'min_samples_leaf':[1,5,10]}
  ]

grid_search = GridSearchCV(estimator = classifier,
                           param_grid = param_grid,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [10]:
print( grid_search.best_score_)
print(grid_search.best_params_)

0.8298091799896854
{'bootstrap': False, 'max_depth': 15, 'max_features': 4, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 20}


In [19]:
from tensorflow.keras import metrics
def top2_acc(labels, logits): 
    return metrics.sparse_top_k_categorical_accuracy(y_true=labels, y_pred=logits, k=2)

In [12]:
from sklearn.model_selection import cross_val_score
accuracies_train = cross_val_score(estimator = grid_search.best_estimator_, X = X_train, y = y_train, cv = 10)
accuracies_test = cross_val_score(estimator = grid_search.best_estimator_, X = X_test, y = y_test, cv = 10)
print('Training accuracy mean',accuracies_train.mean())
print('Training accuracy std',accuracies_train.std())
print('Testing accuracy mean',accuracies_test.mean())
print('Testing accuracy std',accuracies_test.std())

D:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Training accuracy mean 0.829812446898063
Training accuracy std 0.012835497101924173
Testing accuracy mean 0.6250960611760333
Testing accuracy std 0.03492087474450585


In [13]:
import tensorflow as tf
from tensorflow import keras

In [21]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(512, activation="relu"))
model.add(keras.layers.Dense(256, activation="relu"))
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [22]:
model.compile( loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=['accuracy', top2_acc])

In [23]:
X_valid=X_train[:1500]
y_valid=y_train[:1500]

In [24]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [25]:
history = model.fit(X_train, y_train, epochs=100,validation_data=(X_valid, y_valid)) 

Train on 7756 samples, validate on 1500 samples
Epoch 1/100
7756/7756 [==============================] - 3s 376us/sample - loss: 1.5102 - accuracy: 0.4607 - top2_acc: 0.7675 - val_loss: 1.2424 - val_accuracy: 0.4960 - val_top2_acc: 0.8267
Epoch 2/100
7756/7756 [==============================] - 1s 180us/sample - loss: 1.1896 - accuracy: 0.5242 - top2_acc: 0.8493 - val_loss: 1.1549 - val_accuracy: 0.5127 - val_top2_acc: 0.8573
Epoch 3/100
7756/7756 [==============================] - 1s 178us/sample - loss: 1.1336 - accuracy: 0.5379 - top2_acc: 0.8552 - val_loss: 1.1251 - val_accuracy: 0.5080 - val_top2_acc: 0.8587
Epoch 4/100
7756/7756 [==============================] - 1s 178us/sample - loss: 1.1072 - accuracy: 0.5459 - top2_acc: 0.8566 - val_loss: 1.0979 - val_accuracy: 0.5280 - val_top2_acc: 0.8627
Epoch 5/100
7756/7756 [==============================] - 1s 173us/sample - loss: 1.0911 - accuracy: 0.5505 - top2_acc: 0.8580 - val_loss: 1.0777 - val_accuracy: 0.5347 - val_top2_acc: 0.86

7756/7756 [==============================] - 1s 179us/sample - loss: 0.9225 - accuracy: 0.6154 - top2_acc: 0.8880 - val_loss: 0.9146 - val_accuracy: 0.5973 - val_top2_acc: 0.8920
Epoch 44/100
7756/7756 [==============================] - 2s 201us/sample - loss: 0.9207 - accuracy: 0.6159 - top2_acc: 0.8877 - val_loss: 0.9070 - val_accuracy: 0.6087 - val_top2_acc: 0.8900
Epoch 45/100
7756/7756 [==============================] - 1s 185us/sample - loss: 0.9169 - accuracy: 0.6131 - top2_acc: 0.8891 - val_loss: 0.9024 - val_accuracy: 0.6120 - val_top2_acc: 0.8960
Epoch 46/100
7756/7756 [==============================] - 1s 184us/sample - loss: 0.9145 - accuracy: 0.6136 - top2_acc: 0.8887 - val_loss: 0.9001 - val_accuracy: 0.6233 - val_top2_acc: 0.8940
Epoch 47/100
7756/7756 [==============================] - 1s 180us/sample - loss: 0.9108 - accuracy: 0.6182 - top2_acc: 0.8894 - val_loss: 0.8964 - val_accuracy: 0.6273 - val_top2_acc: 0.8960
Epoch 48/100
7756/7756 [=============================

Epoch 86/100
7756/7756 [==============================] - 1s 177us/sample - loss: 0.7986 - accuracy: 0.6748 - top2_acc: 0.9155 - val_loss: 0.7860 - val_accuracy: 0.6567 - val_top2_acc: 0.9127
Epoch 87/100
7756/7756 [==============================] - 1s 178us/sample - loss: 0.7947 - accuracy: 0.6764 - top2_acc: 0.9096 - val_loss: 0.8398 - val_accuracy: 0.6233 - val_top2_acc: 0.9167
Epoch 88/100
7756/7756 [==============================] - 1s 179us/sample - loss: 0.7925 - accuracy: 0.6765 - top2_acc: 0.9123 - val_loss: 0.7670 - val_accuracy: 0.6800 - val_top2_acc: 0.9213
Epoch 89/100
7756/7756 [==============================] - 1s 177us/sample - loss: 0.7917 - accuracy: 0.6739 - top2_acc: 0.9123 - val_loss: 0.7753 - val_accuracy: 0.6720 - val_top2_acc: 0.9200
Epoch 90/100
7756/7756 [==============================] - 1s 181us/sample - loss: 0.7895 - accuracy: 0.6778 - top2_acc: 0.9137 - val_loss: 0.7758 - val_accuracy: 0.6720 - val_top2_acc: 0.9253
Epoch 91/100
7756/7756 [================

In [26]:
model.evaluate(X_test,y_test)

1939/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.052381373085024, 0.5569881, 0.8643631]